# Plan
- Create new WG vcf with AD fix
- Create npy and hdf5 files for release5
    - Create mixtures and crosses specific vcfs
    - Create biallelic versions of the above
    - Create WG npy variants and 2d calldata files for these 4 vcfs
    - Create sites only vcf
    - Create npy WG variants files
    - Create chrom npy variants and calldata files
    - Create HDF5 file

# Setup

In [1]:
%run imports.ipynb
%run _shared_setup.ipynb

docker image: podpearson/biipy_rdp:latest


In [2]:
release5_final_files_dir = '/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0'

In [4]:
mixtures_ids = np.array(["PG%04d-C" % n for n in range(389, 416)])
crosses_fns = ['../../meta/crosses_samples_%s.txt' % x for x in ['3D7xHB3', 'HB3xDd2', '7G8xGB4']]
crosses_samples = (etl
    .fromtsv(crosses_fns[0])
    .cat(etl.fromtsv(crosses_fns[1]))
    .cat(etl.fromtsv(crosses_fns[2]))
)
crosses_ids = crosses_samples.values('sample').array()

In [6]:
crosses_ids

array(['PG0051-C', 'PG0065-C', 'PG0062-C', 'PG0055-C', 'PG0056-C',
       'PG0067-C', 'PG0053-C', 'PG0066-C', 'PG0061-C', 'PG0068-C',
       'PG0069-C', 'PG0070-C', 'PG0071-C', 'PG0072-C', 'PG0063-C',
       'PG0064-C', 'PG0058-C', 'PG0054-C', 'PG0060-C', 'PG0057-C',
       'PG0052-C', 'PG0023-C', 'PG0022-Cx', 'PG0024-C', 'PG0074-C',
       'PG0038-C', 'PG0035-Cx', 'PG0047-C', 'PG0039-C', 'PG0040-Cx',
       'PG0036-C', 'PG0041-C', 'PG0042-C', 'PG0030-C', 'PG0037-C',
       'PG0034-C', 'PG0031-C', 'PG0043-C', 'PG0044-C', 'PG0046-Cx',
       'PG0048-C', 'PG0015-C', 'PG0018-C', 'PG0032-Cx', 'PG0033-Cx',
       'PG0029-Cx', 'PG0008-CW', 'PG0021-C', 'PG0028-C', 'PG0004-CW',
       'PG0017-C', 'PG0016-C', 'PG0045-C', 'PG0026-C', 'PG0025-C',
       'PG0019-C', 'PG0027-C', 'PG0020-C', 'PG0083-C', 'PG0103-CW',
       'PG0112-CW', 'PG0112-C', 'PG0094-CW', 'PG0098-C', 'PG0081-CW',
       'PG0084-C', 'PG0099-C', 'PG0087-C', 'PG0077-CW', 'PG0111-CW',
       'PG0111-C', 'PG0100-CW', 'PG0100-C', 'PG

# Create files

In [3]:
# Need to recreate WG file due to AD Number=R bug
wg_vcf_fn = "%s/SNP_INDEL_WG.combined.filtered.vcf.gz" % (release5_final_files_dir)
!mv {wg_vcf_fn} {wg_vcf_fn + '.old'}
!zcat {wg_vcf_fn + '.old'} \
| sed 's/##FORMAT=<ID=AD,Number=./##FORMAT=<ID=AD,Number=R/' \
| bgzip -c > {wg_vcf_fn}
!bcftools index --tbi {wg_vcf_fn}


In [ ]:
!rm {wg_vcf_fn + '.old'}

In [8]:
# Create mixtures and crosses vcfs
mixtures_samples = ','.join(mixtures_ids)
crosses_samples = ','.join(crosses_ids)

mixtures_vcf_fn = wg_vcf_fn.replace('filtered', 'filtered.mixtures')
mixtures_biallelic_vcf_fn = wg_vcf_fn.replace('filtered', 'filtered.mixtures_biallelic')
crosses_vcf_fn = wg_vcf_fn.replace('filtered', 'filtered.crosses')
crosses_biallelic_vcf_fn = wg_vcf_fn.replace('filtered', 'filtered.crosses_biallelic')

!bcftools view -Oz -o {mixtures_vcf_fn} -s {mixtures_samples} {wg_vcf_fn}
!bcftools view -Ou -s {mixtures_samples} {wg_vcf_fn} | bcftools norm -m -any -Oz -o {mixtures_biallelic_vcf_fn}
!bcftools view -Oz -o {crosses_vcf_fn} -s {crosses_samples} {wg_vcf_fn}
!bcftools view -Ou -s {crosses_samples} {wg_vcf_fn} | bcftools norm -m -any -Oz -o {crosses_biallelic_vcf_fn}

!bcftools index --tbi {mixtures_vcf_fn}
!bcftools index --tbi {mixtures_biallelic_vcf_fn}
!bcftools index --tbi {crosses_vcf_fn}
!bcftools index --tbi {crosses_biallelic_vcf_fn}


Lines   total/split/realigned/skipped:	4212465/2095789/0/0
Error: subset called for sample that does not exist in header: "PG0083-C". Use "--force-samples" to ignore this error.
Error: subset called for sample that does not exist in header: "PG0083-C". Use "--force-samples" to ignore this error.
Failed to open -: unknown file type
[E::hts_open_format] fail to open file '/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0/SNP_INDEL_WG.combined.filtered.crosses.vcf.gz'
Failed to read /nfs/team112_internal/production/release_build/Pf3K/pilot_5_0/SNP_INDEL_WG.combined.filtered.crosses.vcf.gz
[E::hts_open_format] fail to open file '/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0/SNP_INDEL_WG.combined.filtered.crosses_biallelic.vcf.gz'
Failed to read /nfs/team112_internal/production/release_build/Pf3K/pilot_5_0/SNP_INDEL_WG.combined.filtered.crosses_biallelic.vcf.gz


In [11]:
def create_variants_npy(vcf_fn=mixtures_vcf_fn):
    output_dir = '%s.vcfnp_cache' % vcf_fn
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    vcfnp.variants(
        vcf_fn,
        dtypes={
            'REF':                      'a10',
            'ALT':                      'a10',
            'RegionType':               'a25',
            'VariantType':              'a40',
            'RU':                       'a40',
            'set':                      'a40',
            'SNPEFF_AMINO_ACID_CHANGE': 'a20',
            'SNPEFF_CODON_CHANGE':      'a20',
            'SNPEFF_EFFECT':            'a33',
            'SNPEFF_EXON_ID':            'a2',
            'SNPEFF_FUNCTIONAL_CLASS':   'a8',
            'SNPEFF_GENE_BIOTYPE':      'a14',
            'SNPEFF_GENE_NAME':         'a20',
            'SNPEFF_IMPACT':             'a8',
            'SNPEFF_TRANSCRIPT_ID':     'a20',
            'culprit':                  'a14',
        },
        arities={
            'ALT':   6,
            'AF':    6,
            'AC':    6,
            'MLEAF': 6,
            'MLEAC': 6,
            'RPA':   7,
            'ANN':   1,
        },
        fills={
            'VQSLOD': np.nan,
            'QD': np.nan,
            'MQ': np.nan,
            'MQRankSum': np.nan,
            'ReadPosRankSum': np.nan,
            'FS': np.nan,
            'SOR': np.nan,
            'DP': np.nan,
        },
        flatten_filter=True,
        verbose=False,
        cache=True,
        cachedir=output_dir
    )
#     !vcf2npy \
#         --vcf {vcf_fn} \
#         --fasta {GENOME_FN} \
#         --output-dir {output_dir} \
#         --array-type variants \
#         --progress 1000000 \
#         --arity ALT:6 \
#         --arity AF:6 \
#         --arity AC:6 \
#         --arity MLEAF:6 \
#         --arity MLEAC:6 \
#         --arity RPA:7 \
#         --arity ANN:1 \
#         --dtype REF:a10 \
#         --dtype ALT:a10 \
#         --dtype RegionType:a25 \
#         --dtype VariantType:a40 \
#         --dtype RU:a40 \
#         --dtype set:a40 \
#         --dtype SNPEFF_AMINO_ACID_CHANGE:a20 \
#         --dtype SNPEFF_CODON_CHANGE:a20 \
#         --dtype SNPEFF_EFFECT:a33 \
#         --dtype SNPEFF_EXON_ID:a2 \
#         --dtype SNPEFF_FUNCTIONAL_CLASS:a8 \
#         --dtype SNPEFF_GENE_BIOTYPE:a14 \
#         --dtype SNPEFF_GENE_NAME:a20 \
#         --dtype SNPEFF_IMPACT:a8 \
#         --dtype SNPEFF_TRANSCRIPT_ID:a20 \
#         --dtype culprit:a14

def create_calldata_npy(vcf_fn=mixtures_vcf_fn, max_alleles=7):
    output_dir = '%s.vcfnp_cache' % vcf_fn
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    vcfnp.calldata_2d(
        vcf_fn,
        fields=['AD'],
        dtypes={
            'AD': 'u2',
        },
        arities={
            'AD': max_alleles,
        },
#         fills={
#             'AD': np.nan,
#         },
#         flatten_filter=True,
        verbose=False,
        cache=True,
        cachedir=output_dir
    )


In [12]:
# Create mixtures and crosses npy files
# create_variants_npy(mixtures_vcf_fn)
create_calldata_npy(mixtures_vcf_fn)
create_variants_npy(mixtures_biallelic_vcf_fn)
create_calldata_npy(mixtures_biallelic_vcf_fn, 2)
# create_variants_npy(crosses_vcf_fn)
# create_calldata_npy(crosses_vcf_fn)
# create_variants_npy(crosses_biallelic_vcf_fn)
# create_calldata_npy(crosses_biallelic_vcf_fn, 2)


In [13]:
# Create sites only vcf and npy
wg_sites_vcf_fn = wg_vcf_fn.replace('filtered', 'filtered.sites')
!bcftools view --drop-genotypes --output-type z --output-file {wg_sites_vcf_fn} {wg_vcf_fn}
!bcftools index --tbi {wg_sites_vcf_fn}
